# CaDRReS-SC for Drug combination prediction
This notebook show an example of how load a pre-trained CaDRReS-SC model and predict drug response combination for your own data.

In [315]:
import sys, os, pickle
from collections import Counter
import importlib
from ipywidgets import widgets
import pandas as pd
import numpy as np

scriptpath = '..'
sys.path.append(os.path.abspath(scriptpath))

from cadrres import pp, model, evaluation, utility

# Read pre-trained model
We will load and predict using a model trained previously. Please check how to train CaDRReS-SC using your data here [notebook_02_model_prediction.ipynb](./notebook_02_model_prediction.ipynb).

In [316]:
model_dir = '../example_result/'

In [317]:
obj_function = widgets.Dropdown(options=['cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight', 'cadrres'],description='Objetice function')

In [318]:
#choose which model you have trained previously
display(obj_function)

Dropdown(description='Objetice function', options=('cadrres-wo-sample-bias', 'cadrres-wo-sample-bias-weight', …

### Load the pre-trained model based on your selection


In [319]:
model_spec_name = obj_function.value
model_file = model_dir + '{}_param_dict.pickle'.format(model_spec_name)

cadrres_model = model.load_model(model_file)

# Read test data
Again, for this example we load GDSC dataset.
Note: GDSC_exp.tsv can be downloaded from https://www.dropbox.com/s/3v576mspw5yewbm/GDSC_exp.tsv?dl=0

## Notes for other test data

You can apply the model to other gene expression dataset. The input gene expression matrix should have been normalized, i.e. **for each sample, expression values are comparable across genes**. 

In this example the gene expression matrix provided by GDSC is already normalized using RMA.

For RNA-seq data, read count should be normalized by gene length, using normalization methods such as TPM.

In [320]:
gene_exp_df = pd.read_csv('../data/GDSC/GDSC_exp.tsv', sep='\t', index_col=0)
gene_exp_df = gene_exp_df.groupby(gene_exp_df.index).mean()
print("Dataframe shape:", gene_exp_df.shape, "\n")
gene_exp_df.head(2)

Dataframe shape: (17419, 1018) 



,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE,,,,,,,,,,,,,,,,,,,,,
A1BG,6.208447,5.025810,5.506955,4.208349,3.399366,4.917872,3.828088,5.146903,3.107543,5.062066,...,4.272172,3.435025,4.930052,2.900213,4.523712,5.074951,2.957153,3.089628,4.047364,5.329524
A1CF,2.981775,2.947547,2.872071,3.075478,2.853231,3.221491,2.996355,2.893977,2.755668,2.985650,...,2.941659,3.155536,2.983619,3.118312,2.975409,2.905804,2.944488,2.780003,2.870819,2.926353


## Calculate fold-change
We normalized baseline gene expression values for each gene by computing fold-changes compared to the median value across cell-lines

In [321]:
cell_line_log2_mean_fc_exp_df, cell_line_mean_exp_df = pp.gexp.normalize_log2_mean_fc(gene_exp_df)

### Load your clustered gene expression dataset (Patient, pdx, cell line and etc...)
In this example we will load a toy dataset clustered using gene expression. Dimensions: genes x clusters.

In [322]:
cluster_norm_exp_fname = '../data/patient/patient_cluster_tmm.csv'
output_dir = '../result/drug_combination/'

In [323]:
cluster_norm_exp_df = pd.read_csv(cluster_norm_exp_fname, index_col=0).T

In [325]:
cluster_norm_exp_df.head(2)

cluster,A1,A2,B1,B2,C1,C2,D1,D2,E1,E2,F1,F2,F3,G1,H1,H2,I1,I2,J1,K
AAAS,41.834829,29.959785,13.354287,38.641140,24.220163,32.409071,20.358623,11.808539,52.184848,26.108724,16.231304,9.480404,15.701209,51.405674,20.683018,16.910426,17.508102,16.162541,14.853112,43.791031
AAMP,56.531718,25.961475,22.131443,24.830151,37.025232,44.035090,10.663168,16.071073,33.693252,31.331859,50.897945,45.358792,46.604359,53.873082,17.761504,15.493863,18.438727,9.730597,6.787410,60.037445


## Read essential genes list

Or in case you want your training using one specific set of genes.

In [326]:
ess_gene_list = utility.get_gene_list('../data/essential_genes.txt')

In [327]:
selected_gene_list = [g for g in ess_gene_list if g in cluster_norm_exp_df.index]
len(selected_gene_list)

1729

## Calculate kernel feature 

Now we will compute the kernel features between your loaded dataset (e.g, clustered patient data) and cell lines (or any other model you trained previously, e.g. CaDRReS-SC trained on PDX data)

In [328]:
test_kernel_df = pp.gexp.calculate_kernel_feature(cluster_norm_exp_df, cell_line_log2_mean_fc_exp_df, selected_gene_list)

Calculating kernel features based on 1547 common genes
(1729, 20) (17419, 1018)


In [329]:
print("Dataframe shape:", test_kernel_df.shape, "\n")
test_kernel_df.head(2)

Dataframe shape: (20, 1018) 



,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
A1,0.072836,-0.077546,-0.012142,0.004416,-0.001477,0.030616,-0.009137,0.012678,0.027221,-0.036311,...,0.021360,0.013181,-0.046433,0.057867,0.073512,0.001711,0.007254,0.028342,0.012077,0.021961
A2,0.074882,-0.067883,-0.006703,-0.002605,-0.000708,0.025307,-0.014549,0.000004,0.025202,-0.048629,...,0.019706,0.018012,-0.051277,0.057297,0.072076,0.001001,0.000633,0.034680,0.010576,0.021010


# Drug response prediction
Given the pre-trained model we will predict now for your clustered data

In [330]:
print('Predicting drug response using CaDRReS: {}'.format(model_spec_name))
pred_df, P_test_df= model.predict_from_model(cadrres_model, test_kernel_df, model_spec_name)
print('done!')

Predicting drug response using CaDRReS: cadrres-wo-sample-bias
done!


Inspecting the model predictions and save the predictions

In [331]:
#cluster vs drugs
pred_df.head(2)

Drug ID,1,1001,1003,1004,1005,1006,1007,1008,1009,1010,...,64,71,83,86,87,88,89,9,91,94
A1,3.897689,10.962430,-5.786518,-5.639127,3.908731,0.279907,-6.583100,2.435375,7.278870,4.153168,...,2.595250,4.268645,-1.524392,-0.606547,-2.509178,1.189181,3.656169,0.053521,5.708878,5.126888
A2,3.721837,10.932719,-5.831800,-5.736608,3.851029,0.151695,-6.780682,2.425330,7.237191,4.030529,...,2.441818,4.137074,-1.602904,-0.670540,-2.595433,1.173355,3.503584,-0.071620,5.536485,4.956879


In [332]:
print('Saving ' + model_dir + '{}_test_pred.csv'.format(model_spec_name))
pred_df.to_csv(model_dir + '{}_test_pred.csv'.format(model_spec_name))

Saving ../example_result/cadrres-wo-sample-bias_test_pred.csv


# Pre-processing to predict drug combination

In [334]:
model_name = model_spec_name
# for each patient, if cell cluster is less than 5%, then we don't consider that cluster 
freq_cutoff = 0.05
# shift the dosage as GDSC experiment (Syto60) is less sensitive
dosage_shifted = False

In [335]:
drug_info_df = pd.read_csv('../preprocessed_data/GDSC/hn_drug_stat.csv', index_col=0)
drug_info_df.index = drug_info_df.index.astype(str)

drug_id_name_dict = dict(zip(drug_info_df.index, drug_info_df['Drug Name']))
print (drug_info_df.shape)

(81, 27)


In [336]:
drug_info_df.head(2)

,Drug Name,Synonyms,Target,Target Pathway,Selleckchem Cat#,CAS number,PubCHEM,Others,entropy,max_conc,...,median_ic50_9f,log2_median_ic50_9f,log2_median_ic50_hn,median_ic50_hn,median_ic50_3f_hn,log2_median_ic50_3f_hn,median_ic50_9f_hn,log2_median_ic50_9f_hn,num_sensitive,num_sensitive_hn
Drug ID,,,,,,,,,,,,,,,,,,,,,
1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,S1802,2627-69-2,65110,NaN,6.034272,2000.0,...,206.748380,7.691732,9.939784,982.139588,327.379863,8.354822,109.126621,6.769859,476,27
1003,Camptothecin,"7-Ethyl-10-Hydroxy-Camptothecin, SN-38, Irinot...",TOP1,DNA replication,S1288,7689-03-4,104842,"(SN-38, S4908, 86639-52-3) (Irinotecan, S1198,...",4.609530,0.1,...,0.002003,-8.963413,-7.587491,0.005199,0.001733,-9.172454,0.000578,-10.757416,688,30


### Load your pre-trained model results on your own patient data (saved some steps earlier)

In [337]:
cadrres_cluster_df = pd.read_csv(model_dir + '{}_test_pred.csv'.format(model_spec_name), index_col=0)

In [338]:
#load prediction for a certain set of drugs
drug_list = drug_info_df.index
cluster_list = cadrres_cluster_df.index
print(len(drug_list), len(cluster_list))

drug_info_df = drug_info_df.loc[drug_list]
cadrres_cluster_df = cadrres_cluster_df[drug_list]

81 20


In [339]:
if dosage_shifted:
    # Shift by 4 uM
    cadrres_cluster_df = cadrres_cluster_df - 2

### Load patient (or any other model) cluster percentage

In [341]:
freq_df = pd.read_excel('../data/patient/percent_patient_cluster.xlsx', index_col=[0, 1]).reset_index()
freq_df = freq_df.pivot(index='patient_id', columns='cluster', values='percent').fillna(0) / 100

patient_list = freq_df.index

freq_df.head(2)
#show patient by cluster percentage

cluster,A1,A2,B1,B2,C1,C2,D1,D2,E1,E2,F1,F2,F3,G1,H1,H2,I1,I2,J1
patient_id,,,,,,,,,,,,,,,,,,,
Patient_1,0.022857,0.474286,0.0,0.0,0.000000,0.011429,0.0,0.0,0.337143,0.154286,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Patient_2,0.000000,0.000000,0.0,0.0,0.275862,0.235632,0.0,0.0,0.000000,0.000000,0.218391,0.166667,0.103448,0.0,0.0,0.0,0.0,0.0,0.0


### List all pairs of patient and drug

In [342]:
pred_delta_df = pd.DataFrame(cadrres_cluster_df.values - drug_info_df[ref_type].values, columns=drug_list, index=cluster_list)
pred_cv_df = 100 / (1 + (np.power(2, -pred_delta_df)))
pred_kill_df = 100 - pred_cv_df

In [343]:
rows = []
for p in patient_list:
    c_list = freq_df.loc[p][freq_df.loc[p] >= freq_cutoff].index.values
    freqs = freq_df.loc[p][freq_df.loc[p] >= freq_cutoff].values

    print(c_list, freqs)

    p_pred_delta_weighted = np.matmul(pred_delta_df.loc[c_list].values.T, freqs)
    p_pred_delta_mat = pred_delta_df.loc[c_list].values
    
    p_pred_kill_weighted = np.matmul(pred_kill_df.loc[c_list].values.T, freqs)
    p_pred_kill_mat = pred_kill_df.loc[c_list].values

    for d_i, d_id in enumerate(drug_list):
        rows += [[p, d_id] + ['|'.join(c_list)] + ['|'.join(["{:.14}".format(f) for f in freqs])] + 
                 ['|'.join(["{:.14}".format(f) for f in p_pred_delta_mat[:, d_i]])] + 
                 ["{:.14}".format(p_pred_delta_weighted[d_i])] +
                 ['|'.join(["{:.14}".format(f) for f in p_pred_kill_mat[:, d_i]])] + 
                 ["{:.14}".format(p_pred_kill_weighted[d_i])]
                ]

['A2' 'E1' 'E2'] [0.47428571 0.33714286 0.15428571]
['C1' 'C2' 'F1' 'F2' 'F3'] [0.27586207 0.23563218 0.2183908  0.16666667 0.10344828]
['B1' 'B2' 'G1'] [0.2967033  0.23626374 0.46703297]
['H1' 'H2' 'J1'] [0.32236842 0.22368421 0.45394737]
['D1' 'D2'] [0.72527473 0.27472527]
['I1' 'I2'] [0.575 0.425]


In [344]:
single_drug_pred_df = pd.DataFrame(rows, columns=['patient', 'drug_id', 'cluster', 'cluster_p', 'cluster_delta', 'delta', 'cluster_kill', 'kill'])
single_drug_pred_df.head()

,patient,drug_id,cluster,cluster_p,cluster_delta,delta,cluster_kill,kill
0,Patient_1,1001,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,0.99293517368875|1.0329112254328|1.058191058623,0.98243737314013,33.442243435815|32.828334751418|32.443105945183,31.934524663007
1,Patient_1,1003,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.755691477413|1.752643253083|1.8642688369739,1.7112205894598,22.847318138884|22.884583703612|21.547852587259,21.876056365123
2,Patient_1,1004,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.4143740117334|1.4794722838759|1.5497034555972,1.4087080058495,27.281850100904|26.395899870969|25.461038483201,25.766855370335
3,Patient_1,1006,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.4943273294913|1.4668217785019|1.5813190735344,1.4472415329418,26.196336950079|26.566613756126|25.04738172618,25.245745628999
4,Patient_1,1007,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,3.0123161017985|3.1673551646104|3.2265566328267,2.9943612585578,11.027075832071|10.016042902879|9.6522188833902,10.096021343904


In [345]:
if dosage_shifted:
    single_drug_pred_df.to_csv('pred_drug_kill_{}_shifted.csv'.format(model_name), index=False)
else:
    single_drug_pred_df.to_csv('pred_drug_kill_{}.csv'.format(model_name), index=False)

### In case you want to check drug combination for a set of drugs

In [346]:
tested_drug_list = [1032, 1007, 133, 201, 1010] + [182, 301, 302] + [1012]
[drug_id_name_dict[str(d)] for d in tested_drug_list]

['Afatinib',
 'Docetaxel',
 'Doxorubicin',
 'Epothilone B',
 'Gefitinib',
 'Obatoclax Mesylate',
 'PHA-793887',
 'PI-103',
 'Vorinostat']

In [347]:
single_drug_pred_df.loc[:, 'drug_id'] = single_drug_pred_df.loc[:, 'drug_id'].values.astype(str)
single_drug_pred_df.loc[:, 'drug_name'] = [drug_id_name_dict[d] for d in single_drug_pred_df.loc[:, 'drug_id'].values]
patient_list = sorted(list(set(single_drug_pred_df['patient'])))

single_drug_pred_df.head()

,patient,drug_id,cluster,cluster_p,cluster_delta,delta,cluster_kill,kill,drug_name
0,Patient_1,1001,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,0.99293517368875|1.0329112254328|1.058191058623,0.98243737314013,33.442243435815|32.828334751418|32.443105945183,31.934524663007,AICA Ribonucleotide
1,Patient_1,1003,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.755691477413|1.752643253083|1.8642688369739,1.7112205894598,22.847318138884|22.884583703612|21.547852587259,21.876056365123,Camptothecin
2,Patient_1,1004,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.4143740117334|1.4794722838759|1.5497034555972,1.4087080058495,27.281850100904|26.395899870969|25.461038483201,25.766855370335,Vinblastine
3,Patient_1,1006,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.4943273294913|1.4668217785019|1.5813190735344,1.4472415329418,26.196336950079|26.566613756126|25.04738172618,25.245745628999,Cytarabine
4,Patient_1,1007,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,3.0123161017985|3.1673551646104|3.2265566328267,2.9943612585578,11.027075832071|10.016042902879|9.6522188833902,10.096021343904,Docetaxel


### Setup all drug combinations by patient

In [348]:
drug_combi_list = []
n_drugs = len(tested_drug_list)

for p in patient_list:
    for x in range(0, n_drugs-1):
        for y in range(x+1, n_drugs):
            drug_x = str(tested_drug_list[x])
            drug_y = str(tested_drug_list[y])

            drug_combi_list += [[p, drug_x, drug_y]]

drug_combi_df = pd.DataFrame(drug_combi_list, columns=['patient', 'A', 'B'])

print (drug_combi_df.shape)
drug_combi_df.head()

(216, 3)


,patient,A,B
0,Patient_1,1032,1007
1,Patient_1,1032,133
2,Patient_1,1032,201
3,Patient_1,1032,1010
4,Patient_1,1032,182


In [349]:
merge_df = pd.merge(drug_combi_df, single_drug_pred_df, how='left', left_on=['patient', 'A'], right_on=['patient', 'drug_id'])
drug_combi_pred_df = pd.merge(merge_df, single_drug_pred_df[['patient', 'drug_id', 'drug_name', 'cluster_delta', 'delta', 'cluster_kill', 'kill']], how='left', left_on=['patient', 'B'], right_on=['patient', 'drug_id'], suffixes=['_A', '_B'])

### Calculating to what extent drug a, drug b and drug a+b kills any cluster for a given patient

In [350]:
rows = []
for _, data in drug_combi_pred_df.iterrows():
    
    cluster_p = np.array([float(p) for p in data['cluster_p'].split('|')])
    
    cluster_kill_A = np.array([float(k) for k in data['cluster_kill_A'].split('|')])
    cluster_kill_B = np.array([float(k) for k in data['cluster_kill_B'].split('|')])
    
    kill_A = drug_combi_pred_df['kill_A'].astype('float')
    kill_B = drug_combi_pred_df['kill_B'].astype('float')
    
    cluster_kill_C = cluster_kill_A + cluster_kill_B - np.multiply(cluster_kill_A/100, cluster_kill_B/100)*100
    kill_C = np.sum(cluster_p * cluster_kill_C)
    
    best_kill = np.max([kill_A, kill_B])
    improve = kill_C - best_kill
    improve_p = (kill_C - best_kill) / best_kill
    
    ##### specificity (entropy) #####
    
    temp_A = np.sum(cluster_p[cluster_kill_A > cluster_kill_B])
    temp_B = np.sum(cluster_p[cluster_kill_A <= cluster_kill_B])
    if temp_A == 0 or temp_B == 0:
        entropy = 0
    else:
        entropy = -(temp_A * np.log2(temp_A) + temp_B * np.log2(temp_B))
    
    sum_kill_dif = np.sum(np.abs(cluster_kill_A - cluster_kill_B))
    
    ##### save output #####
    
    rows += [['|'.join(["{:.14}".format(k) for k in cluster_kill_C])] + [kill_C, improve, improve_p, entropy, sum_kill_dif]]

In [351]:
drug_combi_pred_df = pd.concat([drug_combi_pred_df, pd.DataFrame(rows, columns=['cluster_kill_C', 'kill_C', 'improve', 'improve_p', 'kill_entropy', 'sum_kill_dif'])], axis=1)
drug_combi_pred_df.shape

(216, 23)

### Final drug combination predictions for patients

In [352]:
drug_combi_pred_df = drug_combi_pred_df[['patient', 'drug_id_A', 'drug_name_A', 'drug_id_B', 'drug_name_B', 'cluster', 'cluster_p', 'cluster_kill_A', 'cluster_kill_B', 'cluster_kill_C', 'kill_A', 'kill_B', 'kill_C', 'improve', 'improve_p', 'kill_entropy', 'sum_kill_dif']]

drug_combi_pred_df.head()

,patient,drug_id_A,drug_name_A,drug_id_B,drug_name_B,cluster,cluster_p,cluster_kill_A,cluster_kill_B,cluster_kill_C,kill_A,kill_B,kill_C,improve,improve_p,kill_entropy,sum_kill_dif
0,Patient_1,1032,Afatinib,1007,Docetaxel,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.7690489417428|1.7824306629234|1.7331750059213,11.027075832071|10.016042902879|9.6522188833902,12.601050405501|11.61994454589|11.218104044108,1.7073725510686,10.096021343904,11.624873,-41.700326,-0.782000,0.0,25.410683
1,Patient_1,1032,Afatinib,133,Doxorubicin,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.7690489417428|1.7824306629234|1.7331750059213,45.66967482903|46.117460785343|45.085836623038,46.630804871512|47.077879686267|46.037595177398,1.7073725510686,44.164747348381,45.091239,-8.233960,-0.154410,0.0,131.588318
2,Patient_1,1032,Afatinib,201,Epothilone B,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.7690489417428|1.7824306629234|1.7331750059213,22.723190870982|24.244264026395|22.582517523058,24.090255445091|25.594557493312|23.924297979562,1.7073725510686,22.435225102693,23.745864,-29.579335,-0.554697,0.0,64.265318
3,Patient_1,1032,Afatinib,1010,Gefitinib,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.7690489417428|1.7824306629234|1.7331750059213,2.3049198711361|2.2394980280562|2.1787004069939,4.0331936522905|3.982011191432|3.8741147220073,1.7073725510686,2.1843636797054,3.853113,-49.472085,-0.927743,0.0,1.438464
4,Patient_1,1032,Afatinib,182,Obatoclax Mesylate,A2|E1|E2,0.47428571428571|0.33714285714286|0.1542857142...,1.7690489417428|1.7824306629234|1.7331750059213,28.178445388398|28.33524023934|27.306166074176,29.449003840198|29.612614891824|28.566077434624,1.7073725510686,27.130609287776,28.358261,-24.966937,-0.468201,0.0,78.535197


In [353]:
if dosage_shifted:
    drug_combi_pred_df.to_csv('pred_combi_kill_{}_shifted.csv'.format(model_name), index=False)
else:
    drug_combi_pred_df.to_csv( 'pred_combi_kill_{}.csv'.format(ref_type, model_name), index=False)

---

**Authors:** [Chayaporn Suphavilai](mailto:@.com), [Rafael Peres da Silva](), Genome Institute of Singapore, Nagarajan Lab, May 5, 2020

---

Reproducibility tips from https://github.com/jupyter-guide/ten-rules-jupyter